In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import os
import matplotlib.pyplot as plt
import datetime as dt
import itertools

In [2]:
#################################### 1. Data Preparation ####################################
# Load the data
base_dir = r"C:\Users\arushi\OneDrive\Desktop\data_fol\15min_data\adanient_data"
df_spot = pd.read_csv(os.path.join(base_dir, "adanient_spot_data.csv"))
df_spot = df_spot.drop(columns=['Unnamed: 0'])


In [3]:
# Calculate EMA for high and low prices
ema_span = 14
df_spot['ema_high'] = df_spot['high'].ewm(span=ema_span, adjust=False).mean().round(2)
df_spot['ema_low'] = df_spot['low'].ewm(span=ema_span, adjust=False).mean().round(2)

# Initialize 'signal' column with a compatible dtype (object)
df_spot['signal'] = 0
df_spot['signal'] = df_spot['signal'].astype('object')

In [4]:
# Find the indices where the close price is greater than the EMA high
buy_indices = df_spot[df_spot['close'] > df_spot['ema_high']].index
sell_indices = df_spot[df_spot['close'] < df_spot['ema_low']].index

# Update the 'signal' column with 'BUY' and 'SELL' at the respective indices
df_spot.loc[buy_indices, 'signal'] = 'BUY'
df_spot.loc[sell_indices, 'signal'] = 'SELL'

# Apply forward fill to propagate the last valid observation forward
df_spot['signal'] = df_spot['signal'].replace(0, None).ffill()
# df_spot['signal'] = df_spot['signal'].shift(1)
# Filter out rows where 'signal' is None or 0
df_spot =df_spot.dropna()

In [5]:
# Load future data
df_future = pd.read_csv(os.path.join(base_dir, "adanient_future_data.csv"))

# Create combined key columns
df_spot['key'] = df_spot['date'].astype(str) + ' ' + df_spot['time'].astype(str)
df_future['key'] = df_future['date'].astype(str) + ' ' + df_future['time'].astype(str)

In [6]:
df_future=df_future.drop(['Unnamed: 0'],axis=1)

In [7]:
df_spot

,Timestamp,open,high,low,close,date,time,ema_high,ema_low,signal,key
3,2018-10-10 10:00:00,135.80,137.95,135.80,137.50,2018-10-10,10:00:00,137.24,135.21,BUY,2018-10-10 10:00:00
4,2018-10-10 10:15:00,137.35,138.50,137.10,138.00,2018-10-10,10:15:00,137.41,135.46,BUY,2018-10-10 10:15:00
5,2018-10-10 10:30:02,138.00,138.60,137.10,137.65,2018-10-10,10:30:00,137.57,135.68,BUY,2018-10-10 10:30:00
6,2018-10-10 10:45:01,137.75,139.40,137.40,138.35,2018-10-10,10:45:00,137.81,135.91,BUY,2018-10-10 10:45:00
7,2018-10-10 11:00:03,138.30,139.90,138.15,139.25,2018-10-10,11:00:00,138.09,136.21,BUY,2018-10-10 11:00:00
...,...,...,...,...,...,...,...,...,...,...,...
33174,2024-02-29 14:15:00,3258.75,3259.70,3255.00,3255.60,2024-02-29,14:15:00,3261.97,3250.62,BUY,2024-02-29 14:15:00
33175,2024-02-29 14:30:00,3255.05,3264.55,3247.25,3262.80,2024-02-29,14:30:00,3262.32,3250.17,BUY,2024-02-29 14:30:00
33176,2024-02-29 14:45:00,3262.90,3268.05,3256.00,3261.55,2024-02-29,14:45:00,3263.08,3250.95,BUY,2024-02-29 14:45:00
33177,2024-02-29 15:00:00,3262.45,3288.00,3260.25,3286.15,2024-02-29,15:00:00,3266.40,3252.19,BUY,2024-02-29 15:00:00


In [8]:
spot_fut_df=pd.merge(df_spot,df_future,on="key").dropna().reset_index(drop=True)

In [9]:
spot_fut_df

,Timestamp_x,open_x,high_x,low_x,close_x,date_x,time_x,ema_high,ema_low,signal,key,Timestamp_y,open_y,high_y,low_y,close_y,Ticker,Expiry,date_y,time_y
0,2018-10-10 10:00:00,135.80,137.95,135.80,137.50,2018-10-10,10:00:00,137.24,135.21,BUY,2018-10-10 10:00:00,2018-10-10 10:00:13,136.40,138.50,136.40,137.75,ADANIENT18OCTFUT,2018-10-25,2018-10-10,10:00:00
1,2018-10-10 10:15:00,137.35,138.50,137.10,138.00,2018-10-10,10:15:00,137.41,135.46,BUY,2018-10-10 10:15:00,2018-10-10 10:15:22,137.55,139.15,137.55,138.65,ADANIENT18OCTFUT,2018-10-25,2018-10-10,10:15:00
2,2018-10-10 10:30:02,138.00,138.60,137.10,137.65,2018-10-10,10:30:00,137.57,135.68,BUY,2018-10-10 10:30:00,2018-10-10 10:30:05,138.75,139.30,137.65,138.15,ADANIENT18OCTFUT,2018-10-25,2018-10-10,10:30:00
3,2018-10-10 10:45:01,137.75,139.40,137.40,138.35,2018-10-10,10:45:00,137.81,135.91,BUY,2018-10-10 10:45:00,2018-10-10 10:45:01,138.25,140.00,138.00,139.05,ADANIENT18OCTFUT,2018-10-25,2018-10-10,10:45:00
4,2018-10-10 11:00:03,138.30,139.90,138.15,139.25,2018-10-10,11:00:00,138.09,136.21,BUY,2018-10-10 11:00:00,2018-10-10 11:00:02,138.80,140.65,138.80,139.80,ADANIENT18OCTFUT,2018-10-25,2018-10-10,11:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32907,2024-02-29 14:15:00,3258.75,3259.70,3255.00,3255.60,2024-02-29,14:15:00,3261.97,3250.62,BUY,2024-02-29 14:15:00,2024-02-29 14:15:01,3253.55,3257.30,3252.55,3253.80,ADANIENT24FEBFUT,2024-02-29,2024-02-29,14:15:00
32908,2024-02-29 14:30:00,3255.05,3264.55,3247.25,3262.80,2024-02-29,14:30:00,3262.32,3250.17,BUY,2024-02-29 14:30:00,2024-02-29 14:30:01,3253.95,3261.90,3247.20,3259.80,ADANIENT24FEBFUT,2024-02-29,2024-02-29,14:30:00
32909,2024-02-29 14:45:00,3262.90,3268.05,3256.00,3261.55,2024-02-29,14:45:00,3263.08,3250.95,BUY,2024-02-29 14:45:00,2024-02-29 14:45:00,3260.00,3266.10,3255.85,3260.40,ADANIENT24FEBFUT,2024-02-29,2024-02-29,14:45:00
32910,2024-02-29 15:00:00,3262.45,3288.00,3260.25,3286.15,2024-02-29,15:00:00,3266.40,3252.19,BUY,2024-02-29 15:00:00,2024-02-29 15:00:00,3263.50,3280.15,3258.75,3277.00,ADANIENT24FEBFUT,2024-02-29,2024-02-29,15:00:00


In [10]:
list_of_trades=[]
sl=5
target=5
tsl=2
trade_monitor={}
for each_spot_fut in spot_fut_df.itertuples():
    if "signal" in trade_monitor:
        if (trade_monitor["signal"]=="BUY" and each_spot_fut.signal=="BUY") or (trade_monitor["signal"]=="SELL" and each_spot_fut.signal=="SELL"):

            if each_spot_fut.signal=="BUY":

                if each_spot_fut.close_y > trade_monitor["entry_price"]:
                    trade_monitor["trailing_stoploss"] = max(trade_monitor["trailing_stoploss"], trade_monitor["entry_price"]-trade_monitor["entry_price"]*tsl/100)
                
                
                if each_spot_fut.low_y<trade_monitor['sl_price']:#stop_loss
                    trade_monitor["exit_date"]=each_spot_fut.date_x
                    trade_monitor["exit_time"]=each_spot_fut.time_x
                    trade_monitor["expiry_out"]=each_spot_fut.Expiry
                    trade_monitor["exit_price"]=trade_monitor['sl_price']
                    trade_monitor["remarks"]="SL"
                    
                    
                elif each_spot_fut.high_y>trade_monitor['target_price']:#target
                    trade_monitor["exit_date"]=each_spot_fut.date_x
                    trade_monitor["exit_time"]=each_spot_fut.time_x
                    trade_monitor["expiry_out"]=each_spot_fut.Expiry
                    trade_monitor["exit_price"]=trade_monitor['target_price']
                    trade_monitor["remarks"]="Target"

                elif each_spot_fut.close_y <= trade_monitor["trailing_stoploss"]:#tsl
                    trade_monitor["exit_date"]=each_spot_fut.date_x
                    trade_monitor["exit_time"]=each_spot_fut.time_x
                    trade_monitor["expiry_out"]=each_spot_fut.Expiry
                    trade_monitor["exit_price"]=trade_monitor["trailing_stoploss"]
                    trade_monitor["remarks"]="Trailing stoploss"

                    
            else:

                if each_spot_fut.close_y < trade_monitor["entry_price"]:
                    trade_monitor["trailing_stoploss"] = min(trade_monitor["trailing_stoploss"], trade_monitor["entry_price"]+trade_monitor["entry_price"]*tsl/100)
                
                if each_spot_fut.high_y>trade_monitor['sl_price']:#stop_loss
                    trade_monitor["exit_date"]=each_spot_fut.date_x
                    trade_monitor["exit_time"]=each_spot_fut.time_x
                    trade_monitor["expiry_out"]=each_spot_fut.Expiry
                    trade_monitor["exit_price"]=trade_monitor['sl_price']
                    trade_monitor["remarks"]="SL"
                    
                elif each_spot_fut.low_y<trade_monitor['target_price']:#target
                    trade_monitor["exit_date"]=each_spot_fut.date_x
                    trade_monitor["exit_time"]=each_spot_fut.time_x
                    trade_monitor["expiry_out"]=each_spot_fut.Expiry
                    trade_monitor["exit_price"]=trade_monitor['target_price']
                    trade_monitor["remarks"]="Target"

                elif each_spot_fut.close_y >= trade_monitor["trailing_stoploss"]:#tsl
                    trade_monitor["exit_date"]=each_spot_fut.date_x
                    trade_monitor["exit_time"]=each_spot_fut.time_x
                    trade_monitor["expiry_out"]=each_spot_fut.Expiry
                    trade_monitor["exit_price"]=trade_monitor["trailing_stoploss"]
                    trade_monitor["remarks"]="Trailing stoploss"
                    
            
        else:#signal change
            trade_monitor["exit_date"]=each_spot_fut.date_x
            trade_monitor["exit_time"]=each_spot_fut.time_x
            trade_monitor["expiry_out"]=each_spot_fut.Expiry
            trade_monitor["exit_price"]=each_spot_fut.close_y
            trade_monitor["remarks"]="Signal Change"
            
        
    else:
        if each_spot_fut.signal=="BUY":
            trade_monitor={}
            trade_monitor["signal"]=each_spot_fut.signal
            trade_monitor["entry_date"]=each_spot_fut.date_x
            trade_monitor["entry_time"]=each_spot_fut.time_x
            trade_monitor["entry_price"]=each_spot_fut.close_y
            trade_monitor["expiry_in"]=each_spot_fut.Expiry
            trade_monitor['sl_price']=trade_monitor["entry_price"]-trade_monitor["entry_price"]*sl/100
            trade_monitor['target_price']=trade_monitor["entry_price"]+trade_monitor["entry_price"]*target/100
            trade_monitor['trailing_stoploss']=trade_monitor["entry_price"]-trade_monitor["entry_price"]*tsl/100
            
        elif each_spot_fut.signal=="SELL":
            trade_monitor={}
            trade_monitor["signal"]=each_spot_fut.signal
            trade_monitor["entry_date"]=each_spot_fut.date_x
            trade_monitor["entry_time"]=each_spot_fut.time_x
            trade_monitor["entry_price"]=each_spot_fut.close_y
            trade_monitor["expiry_in"]=each_spot_fut.Expiry
            trade_monitor['sl_price']=trade_monitor["entry_price"]+trade_monitor["entry_price"]*sl/100
            trade_monitor['target_price']=trade_monitor["entry_price"]-trade_monitor["entry_price"]*target/100
            trade_monitor['trailing_stoploss']=trade_monitor["entry_price"]+trade_monitor["entry_price"]*tsl/100
            
    if "entry_price" in trade_monitor and "exit_price" in trade_monitor:
        if "SELL" ==trade_monitor["signal"]:
            trade_monitor["PNL"]=trade_monitor["entry_price"]-trade_monitor["exit_price"]
        else:
            trade_monitor["PNL"]=trade_monitor["exit_price"]-trade_monitor["entry_price"]
        list_of_trades.append(trade_monitor)
        trade_monitor={}

In [11]:
trades_df=pd.DataFrame(list_of_trades)

In [12]:
trades_df["PNL"].sum()


5420.982500000009

In [13]:
trades_df

,signal,entry_date,entry_time,entry_price,expiry_in,sl_price,target_price,trailing_stoploss,exit_date,exit_time,expiry_out,exit_price,remarks,PNL
0,BUY,2018-10-10,10:00:00,137.75,2018-10-25,130.8625,144.6375,134.995,2018-10-10,15:15:00,2018-10-25,144.6375,Target,6.8875
1,BUY,2018-10-11,09:15:00,139.65,2018-10-25,132.6675,146.6325,136.857,2018-10-11,09:45:00,2018-10-25,138.5000,Signal Change,-1.1500
2,SELL,2018-10-11,10:00:00,138.50,2018-10-25,145.4250,131.5750,141.270,2018-10-11,11:30:00,2018-10-25,142.7500,Signal Change,-4.2500
3,BUY,2018-10-11,11:45:00,143.35,2018-10-25,136.1825,150.5175,140.483,2018-10-11,13:45:00,2018-10-25,140.6000,Signal Change,-2.7500
4,SELL,2018-10-11,14:00:00,140.35,2018-10-25,147.3675,133.3325,143.157,2018-10-12,09:15:00,2018-10-25,142.8500,Signal Change,-2.5000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2398,BUY,2024-02-23,12:30:00,3291.35,2024-02-29,3126.7825,3455.9175,3225.523,2024-02-27,09:15:00,2024-02-29,3319.6500,Signal Change,28.3000
2399,SELL,2024-02-27,09:30:00,3321.20,2024-02-29,3487.2600,3155.1400,3387.624,2024-02-28,09:30:00,2024-02-29,3308.6500,Signal Change,12.5500
2400,BUY,2024-02-28,09:45:00,3298.55,2024-02-29,3133.6225,3463.4775,3232.579,2024-02-28,10:15:00,2024-02-29,3289.4500,Signal Change,-9.1000
2401,SELL,2024-02-28,10:30:00,3285.30,2024-02-29,3449.5650,3121.0350,3351.006,2024-02-29,09:45:00,2024-02-29,3247.0000,Signal Change,38.3000


In [14]:
filtered_df = trades_df
# Initialize mfe% and mae% columns
filtered_df['mfe%'] = 0.0
filtered_df['mae%'] = 0.0

# Add date_time column to df_future for filtering
df_future['date_time'] = pd.to_datetime(df_future['date'] + ' ' + df_future['time'])

for idx, row in filtered_df.iterrows():
    entry_date = row['entry_date']
    entry_time = row['entry_time']
    exit_date = row['exit_date']
    exit_time = row['exit_time']
    trade_type = row['signal']

    # Create timestamp for entry and exit points
    entry_date_time = pd.Timestamp(f"{entry_date} {entry_time}")
    exit_date_time = pd.Timestamp(f"{exit_date} {exit_time}")

    # Filter df_future based on date_time range
    condition = (df_future['date_time'] >= entry_date_time) & (df_future['date_time'] <= exit_date_time)
    temp = df_future[condition]

    if not temp.empty:
        if trade_type == 'BUY':
            mfe = temp['high'].max()
            mae = temp['low'].min()
            # Calculate mfe% and mae%
            filtered_df.at[idx, 'mfe%'] = round((mfe - row['entry_price']), 2)
            filtered_df.at[idx, 'mae%'] = round((mae - row['entry_price']), 2)
        elif trade_type == 'SELL':
            mfe = temp['low'].min()
            mae = temp['high'].max()
            # Calculate mfe% and mae%
            filtered_df.at[idx, 'mfe%'] = round((row['entry_price'] - mfe), 2)
            filtered_df.at[idx, 'mae%'] = round((row['entry_price'] - mae), 2)
    else:
        print(f"No data found between {entry_date_time} and {exit_date_time} for index {idx}.")

In [15]:
filtered_df

,signal,entry_date,entry_time,entry_price,expiry_in,sl_price,target_price,trailing_stoploss,exit_date,exit_time,expiry_out,exit_price,remarks,PNL,mfe%,mae%
0,BUY,2018-10-10,10:00:00,137.75,2018-10-25,130.8625,144.6375,134.995,2018-10-10,15:15:00,2018-10-25,144.6375,Target,6.8875,10.70,-1.35
1,BUY,2018-10-11,09:15:00,139.65,2018-10-25,132.6675,146.6325,136.857,2018-10-11,09:45:00,2018-10-25,138.5000,Signal Change,-1.1500,1.60,-4.50
2,SELL,2018-10-11,10:00:00,138.50,2018-10-25,145.4250,131.5750,141.270,2018-10-11,11:30:00,2018-10-25,142.7500,Signal Change,-4.2500,1.00,-4.60
3,BUY,2018-10-11,11:45:00,143.35,2018-10-25,136.1825,150.5175,140.483,2018-10-11,13:45:00,2018-10-25,140.6000,Signal Change,-2.7500,0.55,-3.55
4,SELL,2018-10-11,14:00:00,140.35,2018-10-25,147.3675,133.3325,143.157,2018-10-12,09:15:00,2018-10-25,142.8500,Signal Change,-2.5000,4.55,-4.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2398,BUY,2024-02-23,12:30:00,3291.35,2024-02-29,3126.7825,3455.9175,3225.523,2024-02-27,09:15:00,2024-02-29,3319.6500,Signal Change,28.3000,58.85,-26.20
2399,SELL,2024-02-27,09:30:00,3321.20,2024-02-29,3487.2600,3155.1400,3387.624,2024-02-28,09:30:00,2024-02-29,3308.6500,Signal Change,12.5500,50.65,-4.80
2400,BUY,2024-02-28,09:45:00,3298.55,2024-02-29,3133.6225,3463.4775,3232.579,2024-02-28,10:15:00,2024-02-29,3289.4500,Signal Change,-9.1000,13.35,-9.10
2401,SELL,2024-02-28,10:30:00,3285.30,2024-02-29,3449.5650,3121.0350,3351.006,2024-02-29,09:45:00,2024-02-29,3247.0000,Signal Change,38.3000,79.85,-7.95
